In [ ]:
pip install numpy==1.26.4 pandas==2.2.2 matplotlib==3.8.4 seaborn==0.13.2 scikit-learn==1.5.0 xlrd==2.0.1 openpyxl==3.1.4

In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
from itertools import accumulate
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, cohen_kappa_score, confusion_matrix
from sklearn.feature_selection import f_classif
from sklearn.utils import resample

In [2]:
df = pd.read_excel('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/X4i8vXLw81g4wEH473zIFA/Diabetes-Classification.xlsx')

In [3]:
df.head()

,Patient number,Cholesterol,Glucose,HDL Chol,Chol/HDL ratio,Age,Gender,Height,Weight,BMI,Systolic BP,Diastolic BP,waist,hip,Waist/hip ratio,Diabetes,Unnamed: 16,Unnamed: 17
0,1,193,77,49,3.9,19,female,61,119,22.5,118,70,32,38,0.84,No diabetes,6.0,6.0
1,2,146,79,41,3.6,19,female,60,135,26.4,108,58,33,40,0.83,No diabetes,NaN,NaN
2,3,217,75,54,4.0,20,female,67,187,29.3,110,72,40,45,0.89,No diabetes,NaN,NaN
3,4,226,97,70,3.2,20,female,64,114,19.6,122,64,31,39,0.79,No diabetes,NaN,NaN
4,5,164,91,67,2.4,20,female,70,141,20.2,122,86,32,39,0.82,No diabetes,NaN,NaN


In [5]:
df.drop(columns=['Unnamed: 16', 'Unnamed: 17'])

,Patient number,Cholesterol,Glucose,HDL Chol,Chol/HDL ratio,Age,Gender,Height,Weight,BMI,Systolic BP,Diastolic BP,waist,hip,Waist/hip ratio,Diabetes
0,1,193,77,49,3.9,19,female,61,119,22.5,118,70,32,38,0.84,No diabetes
1,2,146,79,41,3.6,19,female,60,135,26.4,108,58,33,40,0.83,No diabetes
2,3,217,75,54,4.0,20,female,67,187,29.3,110,72,40,45,0.89,No diabetes
3,4,226,97,70,3.2,20,female,64,114,19.6,122,64,31,39,0.79,No diabetes
4,5,164,91,67,2.4,20,female,70,141,20.2,122,86,32,39,0.82,No diabetes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,386,227,105,44,5.2,83,female,59,125,25.2,150,90,35,40,0.88,No diabetes
386,387,226,279,52,4.3,84,female,60,192,37.5,144,88,41,48,0.85,Diabetes
387,388,301,90,118,2.6,89,female,61,115,21.7,218,90,31,41,0.76,No diabetes
388,389,232,184,114,2.0,91,female,61,127,24.0,170,82,35,38,0.92,Diabetes


In [6]:
frequency_table = df['Diabetes'].value_counts()
props = frequency_table.apply(lambda x: x / len(df['Diabetes']))
print(props)

Diabetes
No diabetes    0.846154
Diabetes       0.153846
Name: count, dtype: float64


In [7]:
df_reduced = df[["Diabetes", "Cholesterol", "Glucose", "BMI", "Waist/hip ratio", "HDL Chol", "Chol/HDL ratio", "Systolic BP", "Diastolic BP", "Weight"]]

numerical_columns = df_reduced.iloc[:, 1:10]

# Applying scaling
scaler = StandardScaler()
preproc_reduced = scaler.fit(numerical_columns)

df_standardized = preproc_reduced.transform(numerical_columns)

# Converting the standardized array back to DataFrame
df_standardized = pd.DataFrame(df_standardized, columns=numerical_columns.columns)

In [8]:
df_standardized.describe()

,Cholesterol,Glucose,BMI,Waist/hip ratio,HDL Chol,Chol/HDL ratio,Systolic BP,Diastolic BP,Weight
count,3.900000e+02,3.900000e+02,3.900000e+02,3.900000e+02,3.900000e+02,3.900000e+02,3.900000e+02,3.900000e+02,3.900000e+02
mean,7.287618e-17,-1.457524e-16,2.277381e-17,-6.741046e-16,4.327023e-17,-6.376666e-17,2.915047e-16,-3.006142e-16,-1.867452e-16
std,1.001285e+00,1.001285e+00,1.001285e+00,1.001285e+00,1.001285e+00,1.001285e+00,1.001285e+00,1.001285e+00,1.001285e+00
min,-2.896986e+00,-1.104399e+00,-2.059272e+00,-2.754229e+00,-2.217470e+00,-1.743891e+00,-2.064517e+00,-2.617764e+00,-1.942901e+00
25%,-6.328534e-01,-4.902078e-01,-7.092421e-01,-7.027598e-01,-7.108267e-01,-7.637287e-01,-6.628646e-01,-6.149262e-01,-6.729533e-01
50%,-9.484179e-02,-3.227011e-01,-1.479938e-01,-1.893664e-02,-2.472441e-01,-1.871623e-01,-4.964184e-02,-9.567210e-02,-1.092203e-01
75%,4.880041e-01,7.659498e-03,5.308134e-01,6.648866e-01,5.060777e-01,5.047173e-01,4.759777e-01,4.977612e-01,5.598254e-01
max,5.285274e+00,5.167799e+00,4.099291e+00,3.536944e+00,4.040895e+00,8.518990e+00,4.943744e+00,3.019853e+00,3.657259e+00


In [9]:
df_stdize = pd.concat([df_reduced['Diabetes'], df_standardized], axis=1)
df_stdize

,Diabetes,Cholesterol,Glucose,BMI,Waist/hip ratio,HDL Chol,Chol/HDL ratio,Systolic BP,Diastolic BP,Weight
0,No diabetes,-0.319013,-0.564655,-0.951944,-0.565995,-0.073401,-0.360132,-0.838071,-0.985822,-1.447312
1,No diabetes,-1.372619,-0.527432,-0.360358,-0.702760,-0.536983,-0.533102,-1.276087,-1.875972,-1.050840
2,No diabetes,0.218998,-0.601879,0.079539,0.117828,0.216339,-0.302476,-1.188484,-0.837464,0.237692
3,No diabetes,0.420753,-0.192418,-1.391841,-1.249818,1.143504,-0.763729,-0.662865,-1.430897,-1.571209
4,No diabetes,-0.969111,-0.304089,-1.300828,-0.839524,0.969660,-1.224982,-0.662865,0.201045,-0.902163
...,...,...,...,...,...,...,...,...,...,...
385,No diabetes,0.443170,-0.043523,-0.542385,-0.018937,-0.363140,0.389404,0.563581,0.497761,-1.298635
386,Diabetes,0.420753,3.194941,1.323387,-0.429231,0.100443,-0.129506,0.300771,0.349403,0.361590
387,No diabetes,2.102039,-0.322701,-1.073295,-1.660112,3.924999,-1.109668,3.542092,0.497761,-1.546430
388,Diabetes,0.555256,1.426814,-0.724411,0.528122,3.693208,-1.455608,1.439613,-0.095672,-1.249076


In [10]:
X = df_stdize.drop(columns=['Diabetes'])
y = df_stdize['Diabetes']

# Split the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Initialize LabelEncoder
label_encoder = LabelEncoder()

y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.fit_transform(y_test)

In [12]:
# Create a KNN classifier
knn = KNeighborsClassifier()

knn.fit(X_train, y_train_encoded)

#calculate overall accuracy
y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test_encoded, y_pred)
print(f'Accuracy: {accuracy:.2%}')

Accuracy: 88.46%


In [13]:
# Create a KNN classifier
knn = KNeighborsClassifier()

param_grid = {'n_neighbors': range(1, 12)}

# Perform grid search with cross-validation
grid_search = GridSearchCV(knn, param_grid, cv=10)
grid_search.fit(X_train, y_train_encoded)


# Best parameters and best score
print("Best parameters found: ", grid_search.best_params_)
print(f"Best accuracy score: , {grid_search.best_score_:.3f}")

# Full results
results = grid_search.cv_results_
for mean_score, std_score, params in zip(results['mean_test_score'], results['std_test_score'], results['params']):
    print(f"Mean accuracy: {mean_score:.3f} (std: {std_score:.3f}) with: {params}")

Best parameters found:  {'n_neighbors': 7}
Best accuracy score: , 0.917
Mean accuracy: 0.875 (std: 0.053) with: {'n_neighbors': 1}
Mean accuracy: 0.820 (std: 0.047) with: {'n_neighbors': 2}
Mean accuracy: 0.901 (std: 0.037) with: {'n_neighbors': 3}
Mean accuracy: 0.897 (std: 0.038) with: {'n_neighbors': 4}
Mean accuracy: 0.913 (std: 0.038) with: {'n_neighbors': 5}
Mean accuracy: 0.913 (std: 0.038) with: {'n_neighbors': 6}
Mean accuracy: 0.917 (std: 0.043) with: {'n_neighbors': 7}
Mean accuracy: 0.917 (std: 0.043) with: {'n_neighbors': 8}
Mean accuracy: 0.917 (std: 0.036) with: {'n_neighbors': 9}
Mean accuracy: 0.917 (std: 0.036) with: {'n_neighbors': 10}
Mean accuracy: 0.913 (std: 0.038) with: {'n_neighbors': 11}


In [14]:
fs_score, fs_p_value = f_classif(X, y)

# Combine scores with feature names
fs_scores = pd.DataFrame({'Feature': X.columns, 'F-Score': fs_score, 'P-Value': fs_p_value})
fs_scores = fs_scores.sort_values(by='F-Score', ascending=False)

print(fs_scores)

           Feature     F-Score       P-Value
1          Glucose  350.809177  3.205119e-56
5   Chol/HDL ratio   31.242678  4.298115e-08
0      Cholesterol   16.893380  4.827353e-05
6      Systolic BP   15.931795  7.853024e-05
3  Waist/hip ratio   12.348083  4.935038e-04
8           Weight   10.588454  1.237749e-03
2              BMI    8.365055  4.040512e-03
4         HDL Chol    5.973355  1.496812e-02
7     Diastolic BP    0.947292  3.310160e-01


In [15]:
# Converting Diabetes column into binary (0 for No Diabetes and 1 for Diabetes)
df_stdize['Diabetes'] = np.where(df_stdize['Diabetes'] == 'Diabetes', 1, 0)
df_stdize

,Diabetes,Cholesterol,Glucose,BMI,Waist/hip ratio,HDL Chol,Chol/HDL ratio,Systolic BP,Diastolic BP,Weight
0,0,-0.319013,-0.564655,-0.951944,-0.565995,-0.073401,-0.360132,-0.838071,-0.985822,-1.447312
1,0,-1.372619,-0.527432,-0.360358,-0.702760,-0.536983,-0.533102,-1.276087,-1.875972,-1.050840
2,0,0.218998,-0.601879,0.079539,0.117828,0.216339,-0.302476,-1.188484,-0.837464,0.237692
3,0,0.420753,-0.192418,-1.391841,-1.249818,1.143504,-0.763729,-0.662865,-1.430897,-1.571209
4,0,-0.969111,-0.304089,-1.300828,-0.839524,0.969660,-1.224982,-0.662865,0.201045,-0.902163
...,...,...,...,...,...,...,...,...,...,...
385,0,0.443170,-0.043523,-0.542385,-0.018937,-0.363140,0.389404,0.563581,0.497761,-1.298635
386,1,0.420753,3.194941,1.323387,-0.429231,0.100443,-0.129506,0.300771,0.349403,0.361590
387,0,2.102039,-0.322701,-1.073295,-1.660112,3.924999,-1.109668,3.542092,0.497761,-1.546430
388,1,0.555256,1.426814,-0.724411,0.528122,3.693208,-1.455608,1.439613,-0.095672,-1.249076


In [16]:
# Number of rows for positive diabetes
positive_diabetes = df_stdize[df_stdize['Diabetes'] == 1].shape[0]
print('Number of rows for positive diabetes: ', positive_diabetes)

# Sample negative cases to match positive cases
negative_diabetes = df_stdize[df_stdize['Diabetes'] == 0]
negative_diabetes_downsampled = resample(negative_diabetes, replace=False, n_samples=positive_diabetes, random_state=42)

# Put positive and negative diabetes case into one df -> balanced
balanced = pd.concat([negative_diabetes_downsampled, df_stdize[df_stdize['Diabetes'] == 1]])
balanced.sample(5)

Number of rows for positive diabetes:  60


,Diabetes,Cholesterol,Glucose,BMI,Waist/hip ratio,HDL Chol,Chol/HDL ratio,Systolic BP,Diastolic BP,Weight
311,1,-0.251762,1.836275,0.580112,0.254593,-0.536983,0.158778,1.790027,0.497761,0.460708
157,1,1.339856,0.477609,1.884635,-0.429231,-0.942618,1.946133,-0.838071,-1.060001,0.658943
82,1,-0.363848,0.887070,2.203182,0.938416,0.448130,-0.706072,0.125565,0.275224,1.476666
233,1,-0.588019,2.748256,2.400377,3.536944,-1.406201,1.427223,0.125565,0.201045,3.533362
9,0,-0.632853,-0.043523,1.020010,-1.933642,0.564026,-0.879042,0.125565,1.239553,-0.183559


In [17]:
balanced['Diabetes'].value_counts()

,count
Diabetes,
0,60
1,60


In [18]:
X_simple = balanced[['Glucose']]
y = balanced['Diabetes']

# Split the data
X_train_simple, X_test_simple, y_train_simple, y_test_simple = train_test_split(X_simple, y, test_size=0.2, random_state=42)


In [19]:
knn_simple = KNeighborsClassifier()
knn_simple.fit(X_train_simple, y_train_simple)
y_pred_simple = knn_simple.predict(X_test_simple)
accuracy = accuracy_score(y_test_simple, y_pred_simple)
print(f'Accuracy: {accuracy:.2%}')

Accuracy: 91.67%


In [20]:
# Evaluate confusion matrix
cm = confusion_matrix(y_test_encoded, y_pred)

# Print confusion matrix
print("Confusion Matrix:")
print(cm)
accuracy = accuracy_score(y_test_encoded, y_pred)
print(f'Accuracy: {accuracy:.2%}')

Confusion Matrix:
[[ 8  8]
 [ 1 61]]
Accuracy: 88.46%
